# Introduction to Reshaping and Permuting Tensors

In this notebook, we will explore two fundamental operations in tensor manipulation: **reshaping** and **permuting**. These operations are crucial in tensor networks, machine learning, and scientific computing, where tensors (multi-dimensional arrays) need to be transformed into different shapes or orders.

## Table of Contents
1. What is a Tensor?
2. Reshaping Tensors
3. Permuting Tensors
4. Array views
5. Exercises


## 1. What is a Tensor?

A **tensor** is a generalization of scalars (0D), vectors (1D), and matrices (2D) to higher dimensions. It is essentially a multi-dimensional array of numbers, and its number of dimensions is referred to as its **order** or sometimes its **rank** although this latter can be confused with the rank of a matrix which is completely different (c.f SVD). For example:

- A **scalar** is a 0D tensor (order-0)
- A **vector** is a 1D tensor (order-1)
- A **matrix** is a 2D tensor (order-2)
- A ND tensor (order-N)

Tensors are widely used in fields like physics, computer science, and machine learning.

## 2. Reshaping Tensors

Reshaping a tensor means changing its shape (the number of elements along each dimension) while preserving the data and the total number of elements. In NumPy this operation will be performed, wherever possible, without changing the order of elements in memory (we will see later where reshapes will lead to copying data).

### Example: Reshaping with NumPy

In [3]:
# Importing numpy
import numpy as np

![](../img/reshape.png)

In [4]:
# Create a 3D tensor (3x3x4 tensor)
tensor = np.arange(36).reshape(3, 3, 4)
print("Original Tensor (3x3x4):")
tensor

Original Tensor (3x3x4):


array([[[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11]],

       [[12, 13, 14, 15],
        [16, 17, 18, 19],
        [20, 21, 22, 23]],

       [[24, 25, 26, 27],
        [28, 29, 30, 31],
        [32, 33, 34, 35]]])

In [5]:
# Reshape the tensor to a shape of (9, 4)
reshaped_tensor = np.reshape(tensor, (9, 4))
print("\nReshaped Tensor (9x4):")

reshaped_tensor


Reshaped Tensor (9x4):


array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11],
       [12, 13, 14, 15],
       [16, 17, 18, 19],
       [20, 21, 22, 23],
       [24, 25, 26, 27],
       [28, 29, 30, 31],
       [32, 33, 34, 35]])

The first two indices of the original tensor have been combined into one (multi) index, the size of this new index is 9 (3x3).
Notice that the total number of elements in the array is the same (36).

### Reshaping and Memory Layout

When we reshape a tensor, NumPy will try to return a "view" of the array. In NumPy, a view of an array is a new array object that points at the same data in memory as the original array, only the way we access the data changes. 

We can see how the elements of an array a arranged in memory by using `numpy.ravel` with `order='K'`. We can see that the above reshape did not change the data in memory:

In [6]:
print("Data in memory of original tensor:")
print(np.ravel(tensor, order='K'))
print("Data in memory of reshaped tensor:")
print(np.ravel(reshaped_tensor, order='K'))

Data in memory of original tensor:
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35]
Data in memory of reshaped tensor:
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35]


So what has changed? Answer: the shape and the strides:

In [7]:
print(f"Shape of orginal tensor: {tensor.shape}, strides of orginal tensor: {tensor.strides}")
print(f"Shape of reshaped tensor: {reshaped_tensor.shape}, strides of reshaped tensor: {reshaped_tensor.strides}")

Shape of orginal tensor: (3, 3, 4), strides of orginal tensor: (96, 32, 8)
Shape of reshaped tensor: (9, 4), strides of reshaped tensor: (32, 8)


The shape attribute should be pretty obvious, we'll come back to what strides means later.

## 3. Permuting Tensors

Permuting a tensor means changing the order of its axes (dimensions). The function for permuting the dimensions of a tensor is called `transpose` in NumPy.

### Example: Permuting with NumPy

![](../img/permute.png)

In [8]:
# Permute the tensor axes
permuted_tensor = np.transpose(tensor, axes=(1, 0, 2))
print("Permuted Tensor (Transpose):")
permuted_tensor

Permuted Tensor (Transpose):


array([[[ 0,  1,  2,  3],
        [12, 13, 14, 15],
        [24, 25, 26, 27]],

       [[ 4,  5,  6,  7],
        [16, 17, 18, 19],
        [28, 29, 30, 31]],

       [[ 8,  9, 10, 11],
        [20, 21, 22, 23],
        [32, 33, 34, 35]]])

### Permuting and Memory Layout

Like `reshape`, `transpose` will avoid copying data, prefering to return a view of the array. In fact, unlike `reshape`, `transpose` will **always** return a view of the array.

## Exercise

1. Confirm that `numpy.transpose` does not change the way the data are stored in memory. 
2. What attributes of the transposed array are different?

## 4. Array views

We have seen how `reshape` and `transpose` usually return views of arrays, instead of copying data. In fact, the same is true of many operations in NumPy - another notable example is slicing. The reason for this choice is of course performance - copying data is **much** slower than just changing a few attributes. *Knowing when functions cause data to be copied is crucial for optimizing performance.*

Above we saw how `numpy.ravel` with `order='K'` can be used to see the underlying layout of the array in memory. A more direct way to tell if data is copied during array operations in NumPy is to use the `base` attribute of an array. If an array is a view of another array (i.e., no data copy was made), its `base` attribute will point to the original array. If the `base` attribute is `None`, it means the array owns its data and a copy was made or a new array was created.

### How to Check for Data Copies:

- If `array.base` is `None`, the data is not a view, and therefore, a copy was made.
- If `array.base` is not `None`, the array is a view of another array, and no data copy was made.

### Example: Checking `base` attribute


In [9]:
# Original array
a = np.array([1, 2, 3, 4])

# Create a view using slicing (no copy)
b = a[:2]
print("View created with slicing:")
print("b.base is a:", b.base is a)  # True, b is a view of a

# Create a copy explicitly
c = a[:2].copy()
print("\nCopy created with .copy():")
print("c.base is None:", c.base is None)  # True, c is a copy and owns its data


View created with slicing:
b.base is a: True

Copy created with .copy():
c.base is None: True


# 5. Exercises

Using reshape and transpose, implement the transformations described by the diagrams. Legs labelled by tuples of indices indicate that these are multi-indices composed of the corresponding indices of the original tensor.

### 1.
![](../img/reshape_ex_1.png)

In [22]:
# 1. Transform the tensor
tensor_1 = np.arange(8).reshape(2, 2, 2).copy()
transformed_tensor_1 = None

In [23]:
# Check answer
expected_tensor_1 = np.array([[0, 1], [2, 3], [4, 5], [6, 7]])
assert (transformed_tensor_1 == expected_tensor_1).all() == True

### 2.
![](../img/reshape_ex_2.png)

In [ ]:
# 2. Transform the tensor
tensor_2 = np.arange(16).reshape(2, 2, 2, 2).copy()
transformed_tensor_2 = None

In [26]:
# Check answer
expected_tensor_2 = np.array([[ 0,  1,  4,  5], [ 2,  3,  6,  7], [ 8,  9, 12, 13], [10, 11, 14, 15]])
assert (transformed_tensor_2 == expected_tensor_2).all() == True

### 3.
![](../img/rehsape_ex_3.png)

In [27]:
# 3. Transform the tensor
tensor_3 = np.arange(32).reshape(2, 2, 2, 2, 2).copy()
transformed_tensor_3 = None

In [28]:
# Check answer
expected_tensor_3 = np.array([[[0,  1],
                               [2,  3],
                               [4,  5],
                               [6,  7],
                               [8,  9],
                               [10, 11],
                               [12, 13],
                               [14, 15]],

                              [[16, 17],
                               [18, 19],
                               [20, 21],
                               [22, 23],
                               [24, 25],
                               [26, 27],
                               [28, 29],
                               [30, 31]]]
                             )
assert (transformed_tensor_3 == expected_tensor_3).all() == True


### 4. 
Which of the above resulting tensors are views of the original tensor, and which are new arrays? 
Hint: check the `base` attribute.